In [21]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection

from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

In [4]:
train = pd.read_csv('train.csv')

In [5]:
id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [6]:
test = pd.read_csv('test.csv')

In [7]:
id='1Zwujaf2YpTLz30dx_seMDcdIfpJ26Jwi'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('features_test_tf_idf.csv')

In [8]:
features_test = pd.read_csv('features_test_tf_idf.csv')

In [9]:
features_test.shape

(3263, 10437)

In [10]:
id='1rOewUmTVGHSU6juiNj5bpBgQBA7PvcGc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('features_train_tf_idf.csv')

In [11]:
features_train = pd.read_csv('features_train_tf_idf.csv')

In [12]:
features_train.shape

(7613, 10437)

In [14]:
y = train.target
x_train, x_validation, y_train, y_validation = train_test_split(features_train, y, test_size=0.3, stratify=y)

#KNN

Buscamos los mejores hiperparametros

In [15]:
knn = KNeighborsClassifier(10)
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
#BUSQUEDA DE HIPERPARAMETROS

#params_knn = {'n_neighbors': np.arange(1, 25)}
#knn_gs = GridSearchCV(knn, params_knn, cv=5)
#knn_gs.fit(x_train, y_train)

#knn_best = knn_gs.best_estimator_
#print(knn_gs.best_params_)

#print('knn score: {}'.format(knn_best.score(x_validation, y_validation)))
#print('knn accuracy_score: {}'.format(knn_best.accuracy_score(x_validation, y_validation)))

In [ ]:
print('knn score: {}'.format(knn.score(x_validation, y_validation)))

knn score: 0.6654991243432574


Realizamos la prediccion con el modelo

In [ ]:
#prediction_knn = knn_best.predict(features_test)
#submission_knn = pd.DataFrame(data={'id':test['id'], 'target': prediction_knn})
#submission_knn.to_csv('tf_idf_knn.csv', index=False)

In [ ]:
prediction_knn = knn.predict(features_test)
submission_knn = pd.DataFrame(data={'id':test['id'], 'target': prediction_knn})
submission_knn.to_csv('tf_idf_knn.csv', index=False)

#NAIVE BAYES CLASSIFICATION

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)} 
model = MultinomialNB()
nb = GridSearchCV(model, parameters, n_jobs=-1, verbose=2)

nb.fit(x_train, y_train)
nb_best = nb.best_estimator_
print(nb.best_params_)

print('naive bayes score: {}'.format(nb_best.score(x_validation, y_validation)))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.1s finished


{'alpha': 0.1}
naive bayes score: 0.6247810858143608


In [ ]:
prediction_nb = nb_best.predict(features_test)

In [ ]:
submission_nb = pd.DataFrame(data={'id':test['id'], 'target': prediction_nb})
submission_nb.to_csv('tf_idf_nb.csv', index=False)

#RED NEURONAL PROFUNDA

In [ ]:
def DNN_model(shape, nClasses, dropout=0.5):
    model = Sequential()
    node = 512
    nLayers = 4
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model_DNN = DNN_model(features_train.shape[1], 2)
model_DNN.fit(features_train, train.target, epochs=10, batch_size=128, verbose=2)

Epoch 1/10
 - 6s - loss: 0.7159 - accuracy: 0.5405
Epoch 2/10
 - 6s - loss: 0.6632 - accuracy: 0.5958
Epoch 3/10
 - 6s - loss: 0.6450 - accuracy: 0.6246
Epoch 4/10
 - 6s - loss: 0.6356 - accuracy: 0.6343
Epoch 5/10
 - 6s - loss: 0.6250 - accuracy: 0.6594
Epoch 6/10
 - 6s - loss: 0.6086 - accuracy: 0.6719
Epoch 7/10
 - 6s - loss: 0.5906 - accuracy: 0.6901
Epoch 8/10
 - 6s - loss: 0.5571 - accuracy: 0.7202
Epoch 9/10
 - 6s - loss: 0.5164 - accuracy: 0.7477
Epoch 10/10
 - 6s - loss: 0.4950 - accuracy: 0.7607


In [ ]:
prediction_dnn = model_DNN.predict_classes(features_test)

In [ ]:
submission_dnn = pd.DataFrame(data={'id':test['id'], 'target': prediction_dnn})
submission_dnn.to_csv('tf_idf_dnn.csv', index=False)

#RANDOM FOREST

GRID SEARCH

In [ ]:
rf = RandomForestClassifier()
params_rf = {'n_estimators': [50, 100, 200]}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
rf_best = rf_gs.best_estimator_
print(rf_gs.best_params_)

{'n_estimators': 200}


In [ ]:
print('rf: {}'.format(rf_best.score(x_validation, y_validation)))

rf: 0.6865148861646234


In [ ]:
prediction_rf = rf_best.predict(features_test)

In [ ]:
submission_rf = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf})
submission_rf.to_csv('tf_idf_rf.csv', index=False)

BAGGING

In [16]:
seed = 7
num_trees = 100

BAGGING CLASSIFIER

In [ ]:
#TARDO 48 MIN Y NO TERMINO

In [ ]:
#kfold = model_selection.KFold(n_splits=10)
#cart = DecisionTreeClassifier()
#model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
#results = model_selection.cross_val_score(model, features_train, train.target, cv=kfold)
#print(results.mean())

In [ ]:
#prediction_bagg = model.predict(features_test)

In [ ]:
#submission_bagg = pd.DataFrame(data={'id':test['id'], 'target': prediction_bagg})
#submission_bagg.to_csv('tf_idf_bagg.csv', index=False)

BAGGING CON RANDOM FOREST

In [18]:
max_features = 3
kfold = model_selection.KFold(n_splits=10)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, features_train, train.target, cv=kfold)
print(results.mean())

0.6038221569215806


In [ ]:
prediction_rf_bagg = model.predict(features_test)

In [ ]:
submission_rf_bagg = pd.DataFrame(data={'id':test['id'], 'target': prediction_rf_bagg})
submission_rf_bagg.to_csv('tf_idf_rf_bagg.csv', index=False)

#NuSCV

TARDO MUCHO. MUY BUENA PREDICCION SOLO CON TFIDF SIN MAS FEATURES.

In [ ]:
#nuscv = NuSVC(probability=True)
#nuscv.fit(x_train, y_train)

#print('rf: {}'.format(nuscv.score(x_validation, y_validation)))

#prediction_nuscv = nuscv.predict(features_test)

#submission_nuscv = pd.DataFrame(data={'id':test['id'], 'target': prediction_nuscv})
#submission_nuscv.to_csv('tf_idf_nuscv.csv', index=False)

#XGBoost

In [19]:
from sklearn.metrics import accuracy_score

In [22]:
model = XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [23]:
y_pred = model.predict(x_validation)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_validation, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))

Accuracy: 69.70228%


In [ ]:
submission_xgboost = pd.DataFrame(data={'id':test['id'], 'target': prediction_xgboost})
submission_xgboost.to_csv('tf_idf_xgboost.csv', index=False)